In [2]:
import syft as sy
hook = sy.TorchHook()
from torch import nn, optim

# A Toy Dataset
data1 = sy.Var(sy.FloatTensor([[0,0],[0,1],[1,0],[1,1]]))
print(data1)

Variable containing:
 0  0
 0  1
 1  0
 1  1
[syft.core.frameworks.torch.tensor.FloatTensor of size 4x2]



In [3]:
# A Toy Dataset
data = sy.Var(sy.FloatTensor([[0,0],[0,1],[1,0],[1,1]]))
target = sy.Var(sy.FloatTensor([[0],[0],[1],[1]]))
# A Toy Model
model = nn.Linear(2,1)
def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data[0])

In [4]:
train()

2.480665683746338
0.3744840621948242
0.1490488201379776
0.08914659917354584
0.05702805146574974
0.03683601692318916
0.023860502988100052
0.01549103669822216
0.010082890279591084
0.006582002155482769
0.004311083350330591
0.00283449562266469
0.0018717320635914803
0.0012419970007613301
0.000828591815661639
0.0005560780991800129
0.0003756026562768966
0.00025545910466462374
0.00017502046830486506
0.00012082819011993706


In [5]:
# create a couple workers
bob = sy.VirtualWorker(id="bob")
alice = sy.VirtualWorker(id="alice")
# get pointers to training data on each worker by
# sending some training data to bob and alice
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)
data_alice = data[2:].send(alice)
target_alice = target[2:].send(alice)

# organize pointers into a list
datasets = [(data_bob,target_bob),(data_alice,target_alice)]

# Iniitalize A Toy Model
model = nn.Linear(2,1)

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):
        
        # NEW) iterate through each worker's dataset
        for data,target in datasets:
            
            # NEW) send model to correct worker
            model.send(data.location)

            # 1) erase previous gradients (if they exist)
            opt.zero_grad()

            # 2) make a prediction
            pred = model(data)

            # 3) calculate how much we missed
            loss = ((pred - target)**2).sum()

            # 4) figure out which weights caused us to miss
            loss.backward()

            # NEW) get model (with gradients)
            model.get()

            # 5) change those weights
            opt.step()

            # 6) print our progress
            print(loss.get().data[0]) # NEW) slight edit... need to call .get() on loss

In [6]:
train()

0.3257891833782196
1.0029733180999756
0.6419966220855713
0.2518211007118225
0.4571426510810852
0.16013969480991364
0.31190192699432373
0.11161743104457855
0.21451550722122192
0.07930319011211395
0.14908535778522491
0.057053592056035995
0.10462719947099686
0.041462622582912445
0.07407577335834503
0.030377252027392387
0.05285777151584625
0.0223979614675045
0.03797813132405281
0.016596117988228798
0.027451517060399055
0.012343203648924828
0.01994597539305687
0.009205787442624569
0.014557414688169956
0.006879961583763361
0.010665345005691051
0.005149348638951778
0.007839417085051537
0.003858091775327921
0.005778315477073193
0.0028927046805620193
0.004269201774150133
0.002169900806620717
0.0031605870462954044
0.0016281919088214636
0.002343868836760521
0.0012219188502058387
0.001740740262903273
0.0009170923149213195
